In [13]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Overview

## Introduction

### Requirements

### Motivation

### Learning Objectives

* Tools
    * Pandas
        * For Data storage
        * Also for "scraping"
    * Requests/BeautifulSoup4/LXML
        * For scraping (DUH)

# Land Acknowledgement

# Ethics Of Web-Scraping

## Am I allowed to take this data? Can I use it in my research? What can I use it for?

* Site TOS
* robots.txt

## Can the site handle my requests?

* Speed considerations
* Small site?

# Actual Tutorial

# Background: Data Types & Pandas

## Data Types

### list

* Store multiple items (elements) in a single variable. 
* Elements are separated by commas.
* 

Note that indices start at 0 in Python!

In [2]:
# initialize a list
ex_list1 = [0, 1, 2]

# print the list
print("The 1st element of ex_list1 is", ex_list1[0])

# print length of a list
print("List Length:", len(ex_list1))

# lists can contain 
ex_list2 = ["Three", [4,5]]

# add two lists together 
ex_added_list = ex_list1 + ex_list2
print(ex_added_list)

The 1st element of ex_list1 is 0
List Length: 3
[0, 1, 2, 'Three', [4, 5]]


### Dict

### Set

## Pandas

### Series

### DataFrame

# Web Scraping

## Import Necessary packages

In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

import lxml
import lxml.html
import cssselect
from datetime import datetime
from selenium import webdriver


## Pandas read html

### USDA FIPS

In [4]:
# import data from USDA. Output is a LIST of tables
usda_fips_page = pd.read_html("https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697")

In [5]:
# print output to see what we have...
usda_fips_page

[                                                   FIPS        Name State
 0                                                 01001     Autauga    AL
 1                                                 01003     Baldwin    AL
 2                                                 01005     Barbour    AL
 3                                                 01007        Bibb    AL
 4                                                 01009      Blount    AL
 ...                                                 ...         ...   ...
 3228                                              72153       Yauco    PR
 3229                                              78010   St. Croix    VI
 3230                                              78020    St. John    VI
 3231                                              78030  St. Thomas    VI
 3232  //<![CDATA[ var $s = {} var isClick = false; $...         NaN   NaN
 
 [3233 rows x 3 columns],
                                                    FIPS        Name Sta

In [6]:
# Strange output!
usda_fips_page[0]

,FIPS,Name,State
0,01001,Autauga,AL
1,01003,Baldwin,AL
2,01005,Barbour,AL
3,01007,Bibb,AL
4,01009,Blount,AL
...,...,...,...
3228,72153,Yauco,PR
3229,78010,St. Croix,VI
3230,78020,St. John,VI
3231,78030,St. Thomas,VI


In [7]:
# looks like the first few dataframes are identical... this is strange but we only need one so it isn't a problem!
usda_fips_page[1]

,FIPS,Name,State
0,01001,Autauga,AL
1,01003,Baldwin,AL
2,01005,Barbour,AL
3,01007,Bibb,AL
4,01009,Blount,AL
...,...,...,...
3228,72153,Yauco,PR
3229,78010,St. Croix,VI
3230,78020,St. John,VI
3231,78030,St. Thomas,VI


In [8]:
# make variable for fips table we want
usda_fips = usda_fips_page[0]

In [21]:
# let's examine the dataframe more closely to make sure everything is correct

# change display options to show all rows
pd.set_option('display.max_rows', None)

usda_fips

,FIPS,Name,State
0,01001,Autauga,AL
1,01003,Baldwin,AL
2,01005,Barbour,AL
3,01007,Bibb,AL
4,01009,Blount,AL
5,01011,Bullock,AL
6,01013,Butler,AL
7,01015,Calhoun,AL
8,01017,Chambers,AL
9,01019,Cherokee,AL


In [9]:
# it looks like the last row is the only incorrect one, so let's delete it
usda_fips.tail()

,FIPS,Name,State
3228,72153,Yauco,PR
3229,78010,St. Croix,VI
3230,78020,St. John,VI
3231,78030,St. Thomas,VI
3232,//<![CDATA[ var $s = {} var isClick = false; $...,NaN,NaN


In [10]:
# Let's confirm there are no other rows we need to drop. Check for nan values in any col pt. 1
usda_fips.isnull()

,FIPS,Name,State
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
3228,False,False,False
3229,False,False,False
3230,False,False,False
3231,False,False,False


In [11]:
# Check for nan values in any col pt. 2
usda_fips.isnull().sum()

FIPS     0
Name     1
State    1
dtype: int64

In [17]:
# drop the last row

# this is the output we want
usda_fips.drop(3232)

# now replace the existing "usda_fips" dataframe with the version missing the last row (inplace=True)
usda_fips.drop(3232, inplace=True)

,FIPS,Name,State
0,01001,Autauga,AL
1,01003,Baldwin,AL
2,01005,Barbour,AL
3,01007,Bibb,AL
4,01009,Blount,AL
...,...,...,...
3227,72151,Yabucoa,PR
3228,72153,Yauco,PR
3229,78010,St. Croix,VI
3230,78020,St. John,VI


### Wikipedia FIPS

In [20]:
# import data from wikipedia. Again, output is a LIST of tables
fips_page = pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county")

In [25]:
# let's print the output and take a look...
fips_page

[    0                                                  1
 0 NaN  This article may be too long to read and navig...,
        FIPS County or equivalent State or equivalent
 0      1001       Autauga County             Alabama
 1      1003       Baldwin County             Alabama
 2      1005       Barbour County             Alabama
 3      1007          Bibb County             Alabama
 4      1009        Blount County             Alabama
 ...     ...                  ...                 ...
 3237  56037    Sweetwater County             Wyoming
 3238  56039         Teton County             Wyoming
 3239  56041         Uinta County             Wyoming
 3240  56043      Washakie County             Wyoming
 3241  56045        Weston County             Wyoming
 
 [3242 rows x 3 columns],
   .mw-parser-output .navbar{display:inline;font-size:88%;font-weight:normal}.mw-parser-output .navbar-collapse{float:left;text-align:left}.mw-parser-output .navbar-boxtext{word-spacing:0}.mw-parser-output .

In [26]:
# first element?
fips_page[0]

,0,1
0,NaN,This article may be too long to read and navig...


In [27]:
fips_page[1]

,FIPS,County or equivalent,State or equivalent
0,1001,Autauga County,Alabama
1,1003,Baldwin County,Alabama
2,1005,Barbour County,Alabama
3,1007,Bibb County,Alabama
4,1009,Blount County,Alabama
...,...,...,...
3237,56037,Sweetwater County,Wyoming
3238,56039,Teton County,Wyoming
3239,56041,Uinta County,Wyoming
3240,56043,Washakie County,Wyoming


In [ ]:
# we want the 2nd table (recall: indices start at 0)
fips = fips_page[1]

# remove all hyperlinks (these look like "... County [h]", etc.)
fips['County or equivalent'] = fips['County or equivalent'].str.replace(r"\[.*\]","")

# convert to uppercase
fips['County or equivalent'] = fips['County or equivalent'].apply(lambda x: x.upper())
fips['State or equivalent'] = fips['State or equivalent'].apply(lambda x: x.upper())

# replace "St." with "Saint"
fips['County or equivalent'] = [x.replace('ST.','SAINT') for x in fips['County or equivalent']]

# remove everything after a comma in a county name (e.g. "ANCHORAGE, MUNICIPALITY OF")
fips['County or equivalent'] = [x.split(',')[0] for x in fips['County or equivalent']]

# replace DC info to correspond to GeoCov19 format
dc_loc = fips[fips['County or equivalent']=='DISTRICT OF COLUMBIA'].index.tolist()[0]
fips['State or equivalent'].loc[dc_loc] = 'WASHINGTON, D.C.'
fips['County or equivalent'].loc[dc_loc] = 'WASHINGTON'

############################################################################################

# Save FIPS dataframe to file

FIPS_SAVE_PATH = 

fips.to_csv(FIPS_SAVE_PATH, index=False)

## Request  +BeautifulSoup

### Red-bellied Snake (Wikipedia) Text Analysis

Hypothetical: we want to get a list of all words, and their frequency, from this wikipedia page

In [32]:
wiki_snake_url = "https://en.wikipedia.org/wiki/Red-bellied_black_snake"
wiki_snake_response = requests.get(wiki_snake_url)

# printing "200" means that the page was successfully downloaded!
print(wiki_snake_response)

<Response [200]>


In [33]:
#let's examine the output... which turns out to be a mess (this is where BeautifulSoup comes in handy)
wiki_snake_response.text

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Red-bellied black snake - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"bb99608c-73aa-4603-b6fe-a20f5bf1b73d","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Red-bellied_black_snake","wgTitle":"Red-bellied black snake","wgCurRevisionId":1033489125,"wgRevisionId":1033489125,"wgArticleId":2290687,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles containing Ancient Greek (to 1453)-language text","CS1 French-language sources (fr)","CS1 German-language sources (de)","C

In [34]:
wiki_snake_soup = BeautifulSoup(wiki_snake_response.text, 'html.parser')

# BeautifulSoup gives us a more readable format (barely)
wiki_snake_soup.prettify()

'<!DOCTYPE html>\n<html class="client-nojs" dir="ltr" lang="en">\n <head>\n  <meta charset="utf-8"/>\n  <title>\n   Red-bellied black snake - Wikipedia\n  </title>\n  <script>\n   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"bb99608c-73aa-4603-b6fe-a20f5bf1b73d","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Red-bellied_black_snake","wgTitle":"Red-bellied black snake","wgCurRevisionId":1033489125,"wgRevisionId":1033489125,"wgArticleId":2290687,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles containing Ancient Greek (to 1453)-language text","CS1 French-language sources (fr)","CS1 German-lang

In [35]:
# let's make this even more readable, so we can identify parts of the text we want to gather

# this shows us all html with an "p" tag (paragraphs), but we just want the text
for paragraph in wiki_snake_soup.find_all('p'):
    print(paragraph.text)
    





The red-bellied black snake (Pseudechis porphyriacus) is a species of venomous snake in the family Elapidae, indigenous to Australia. Originally described by George Shaw in 1794 as a species new to science, it is one of eastern Australia's most commonly encountered snakes. Averaging around 1.25 m (4 ft 1 in) in length, it has glossy black upperparts, bright red or orange flanks, and a pink or dull red belly. It is not aggressive and generally retreats from human encounters, but can attack if provoked.  Although its venom can cause significant illness, no deaths have been recorded from its bite, which is less venomous than other Australian elapid snakes. The venom contains neurotoxins, myotoxins, and coagulants and has haemolytic properties. Victims can also lose their sense of smell.

Common in woodlands, forests and swamplands, the red-bellied black snake often ventures into nearby urban areas. It forages in bodies of shallow water, commonly with tangles of water plants and logs, 

In [36]:
[paragraph.text for paragraph in wiki_snake_soup.find_all('p')]

['\n',
 '\n',
 "The red-bellied black snake (Pseudechis porphyriacus) is a species of venomous snake in the family Elapidae, indigenous to Australia. Originally described by George Shaw in 1794 as a species new to science, it is one of eastern Australia's most commonly encountered snakes. Averaging around 1.25\xa0m (4\xa0ft 1\xa0in) in length, it has glossy black upperparts, bright red or orange flanks, and a pink or dull red belly. It is not aggressive and generally retreats from human encounters, but can attack if provoked.  Although its venom can cause significant illness, no deaths have been recorded from its bite, which is less venomous than other Australian elapid snakes. The venom contains neurotoxins, myotoxins, and coagulants and has haemolytic properties. Victims can also lose their sense of smell.\n",
 'Common in woodlands, forests and swamplands, the red-bellied black snake often ventures into nearby urban areas. It forages in bodies of shallow water, commonly with tangles 

In [37]:
# we're almost there, we just need to combine this output

wiki_snake_text = ' '.join([paragraph.text for paragraph in wiki_snake_soup.find_all('p')])

In [38]:
wiki_snake_text
# looks good, but we should remove '\n'

wiki_snake_text = wiki_snake_text.replace('\n','')

In [39]:
wiki_snake_text

'  The red-bellied black snake (Pseudechis porphyriacus) is a species of venomous snake in the family Elapidae, indigenous to Australia. Originally described by George Shaw in 1794 as a species new to science, it is one of eastern Australia\'s most commonly encountered snakes. Averaging around 1.25\xa0m (4\xa0ft 1\xa0in) in length, it has glossy black upperparts, bright red or orange flanks, and a pink or dull red belly. It is not aggressive and generally retreats from human encounters, but can attack if provoked.  Although its venom can cause significant illness, no deaths have been recorded from its bite, which is less venomous than other Australian elapid snakes. The venom contains neurotoxins, myotoxins, and coagulants and has haemolytic properties. Victims can also lose their sense of smell. Common in woodlands, forests and swamplands, the red-bellied black snake often ventures into nearby urban areas. It forages in bodies of shallow water, commonly with tangles of water plants an

In [40]:
# now let's count occurences of each word
# import nltk

nltk.FreqDist(wiki_snake_text)

FreqDist({' ': 2352, 'e': 1481, 'a': 1045, 'n': 876, 't': 845, 's': 814, 'i': 775, 'r': 738, 'o': 729, 'l': 602, ...})

In [41]:
# looks like we need to split the text first:
split_wiki_snake_text = wiki_snake_text.split(' ')

In [43]:
wiki_snake_word_freqs = nltk.FreqDist(split_wiki_snake_text)

In [45]:
#what are we working with?
type(wiki_snake_word_freqs)

nltk.probability.FreqDist

In [50]:
wiki_snake_freqs

[]

In [51]:
wiki_snake_words = []
wiki_snake_freqs = []

for word in wiki_snake_word_freqs:
    wiki_snake_words.append(word)
    wiki_snake_freqs.append(wiki_snake_word_freqs[word])

print(wiki_snake_words)
print(wiki_snake_freqs)

['and', 'the', 'in', 'to', 'of', 'black', 'a', 'red-bellied', 'snake', 'is', 'The', 'or', 'are', 'as', 'it', 'can', 'on', 'from', 'snakes', 'have', 'with', 'be', 'has', 'was', 'their', 'species', 'been', 'which', 'they', 'by', 'It', 'for', 'may', 'its', 'found', 'that', '', 'most', 'commonly', 'around', 'not', 'generally', 'other', 'also', 'South', 'In', 'at', 'They', 'red', 'but', 'venom', 'than', 'into', 'water', 'an', 'up', 'when', 'active', 'described', 'eastern', 'recorded', 'less', 'New', 'genus', 'even', 'were', 'A', 'one', 'pink', 'prey', 'snake,', 'under', 'more', 'Its', 'early', 'body', 'brown', 'head', 'scales', 'his', 'Australia', 'bite', 'venomous', 'belly.', 'Although', 'Australian', 'elapid', 'contains', 'urban', 'bodies', 'logs,', 'where', 'main', 'frogs,', 'small', 'first', 'he', 'name', 'Snake', 'two', 'P.', 'p.', 'rarely', 'some', 'Snakes', 'average', 'snakes,', 'female', 'areas', 'Red-bellied', 'reported', 'preferred', 'though', 'between', 'late', 'males', 'out', 'l

In [53]:
# finally, we convert it to a pandas dataframe so we can easily save it in .csv format
wiki_snake_output = pd.DataFrame({'Word': wiki_snake_words, 'Frequency': wiki_snake_freqs})

In [54]:
wiki_snake_output

,Word,Frequency
0,and,83
1,the,76
2,in,62
3,to,60
4,of,50
...,...,...
1084,fish,1
1085,"fillets,",1
1086,"chicken,",1
1087,dog,1


In [55]:
wiki_snake_output.to_csv(r"Red Bellied Snake Words & Frequencies.csv")

#